In [1]:
import imageToText

ModuleNotFoundError: No module named 'imageToText'

In [ ]:
import sys

from PyQt5.QtCore import QSize, Qt
from PyQt5.QtWidgets import QWidget ,QApplication,QLabel,QMainWindow, QPushButton, QDesktopWidget, QVBoxLayout, QHBoxLayout,QLineEdit
from PyQt5.QtGui import QPixmap,QImage

from mss import mss
import numpy as np
from PIL import Image
import cv2



class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.text = "blank"
        self.monitorX = 0
        self.monitorY = 0
        self.monitorWidth = 1000
        self.monitorHeight = 1000
        
        
        # Ana Ekran Özellikleri
        desktop = QDesktopWidget()
        self.screenWidth = int(desktop.screenGeometry().width()*0.3)
        self.screenHeight = int(desktop.screenGeometry().height()*0.8)

        self.setWindowTitle("Ekran Kaşifi")
        self.setFixedSize(QSize(self.screenWidth,self.screenHeight))
        
        ## Widgetleri Ekle
        widget = QWidget()
        widget.setLayout(self.compenents())
        self.setCentralWidget(widget)
        

    def compenents(self):
        column1 = QVBoxLayout()
        row1 = QHBoxLayout()
        row2 = self.text_fields()   
        row3 = QHBoxLayout()
        row4 = QHBoxLayout()
        
        row1.addWidget(self.monitor())
        row3.addWidget(self.processed_text())
        row4.addWidget(self.button_process())
        row4.addWidget(self.button_copy_text())
        
        column1.addLayout(row1)
        column1.addLayout(row2)
        column1.addLayout(row3)
        column1.addLayout(row4)
        return column1
        

    # Çekilen resim monitoru
    def monitor(self):
        self.lbl= QLabel()
        pixmap = QPixmap()
        self.lbl.setPixmap(pixmap)
        self.lbl.setFixedSize(QSize(int(self.screenWidth*0.8),int(self.screenHeight*0.3)))

        return self.lbl
    
    
    # tahmin edilen yazı kısmı
    def processed_text(self):
        lbl = QLabel(self.text)
        lbl.setFixedSize(QSize(int(self.screenWidth),int(self.screenHeight*0.3)))
        return lbl
    
    # işlenecek ekranın boyutu
    def text_fields(self):
        labelx = QLabel("X")
        labelx.setFixedSize(QSize(34,34))
        line_editx = QLineEdit()
        line_editx.setFixedSize(QSize(34,34))
 
        labely = QLabel("Y")
        labely.setFixedSize(QSize(34,34))
        line_edity = QLineEdit()
        line_edity.setFixedSize(QSize(34,34))
    
        labelwidth = QLabel("Genişlik")
        labelwidth.setFixedSize(QSize(34,34))
        line_editwidth = QLineEdit()
        line_editwidth.setFixedSize(QSize(34,34))

        labelheight = QLabel("YÜkseklik")  
        labelheight.setFixedSize(QSize(60,34))
        line_editheight = QLineEdit()
        line_editheight.setFixedSize(QSize(34,34))
        

        
        def do_action():
            self.monitorY= int(line_editx.text())
            self.monitorX= int(line_edity.text())
            self.monitorWidth= int(line_editwidth.text())
            self.monitorHeight= int(line_editheight.text())
            
        button = QPushButton("Uygula")
        button.setFixedSize(QSize(100,34))
        button.clicked.connect(do_action)
        lay = QHBoxLayout()
        
        lay.addWidget(labelx)
        lay.addWidget(line_editx)
        lay.addWidget(labely)
        lay.addWidget(line_edity)        
        lay.addWidget(labelwidth)
        lay.addWidget(line_editwidth)        
        lay.addWidget(labelheight)
        lay.addWidget(line_editheight)
        lay.addWidget(button)
        return lay
    

    ## Resmi çek ve işle
    def button_process(self):
        button = QPushButton("Çek")
        button.clicked.connect(self.take_picture)
        button.setFixedHeight(int(self.screenHeight*0.2))
        return button
    
    def take_picture(self):
        ## resmi çek
        mon = {
            "top" : self.monitorX,
            "left":self.monitorY,
            "width":self.monitorWidth,
            "height":self.monitorHeight}
        sct = mss()
        im = sct.grab(mon)
        im = Image.frombytes("RGB",im.size,im.rgb)
        im = np.array(im)
        
        
        ## resmi programda göster
        image_rgb = cv2.cvtColor(cv2.resize(im,(int(self.screenWidth*0.8),int(self.screenHeight*0.3))), cv2.COLOR_BGR2RGB)
        qimage = QImage(image_rgb, image_rgb.shape[1], image_rgb.shape[0], QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        self.lbl.setPixmap(pixmap)
        
        return im
    
    ## yazıyı koypyala
    def button_copy_text(self):
        button = QPushButton("Yazıyı Kopyala")
        button.clicked.connect(self.copy_text)
        button.setFixedHeight(int(self.screenHeight*0.2))
        return button
    
    def copy_text(self):
        clipboard = QApplication.clipboard()
        clipboard.setText(self.text)
    
app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()